In [1]:
import os
import json
import numpy as np

import torch
from tqdm import tqdm

from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

import random
import time


from qwen_vl_utils import process_vision_info
from io import BytesIO
from PIL import Image
import base64
import io
from openai import OpenAI
import requests

/cpfs/user/zhengziwei/ENV/miniconda3/envs/verl_agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/cpfs/user/zhengziwei/ENV/miniconda3/envs/verl_agent/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [3]:
openai_api_key = "EMPTY"
openai_api_base = "http://10.39.11.28:10000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
response = requests.get(f"{openai_api_base}/models")
models = response.json()
model_name = models['data'][0]['id']

In [4]:
vstar_bench_path = '/cpfs/user/honglingyi/DATA/LLM/Vstar/vstar_bench'
test_types = ['direct_attributes', 'relative_position']
per_type_acc = {}
for test_type in test_types:
    per_type_acc[test_type] = []
all_acc = []
abc_map = {1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F'}

In [5]:
def get_chat_template():
    chat_template = """
Below are two answers to a question. Question is [Question], [Standard Answer] is the standard answer to the question, and [Model_answer] is the answer extracted from a model's output to this question.  Determine whether these two answers are consistent.
Note that [Model Answer] is consistent with [Standard Answer] whenever they are essentially the same. If the meaning is expressed in the same way, it is considered consistent, for example, 'pink' and 'it is pink'.
If they are consistent, Judement is 1; if they are different, Judement is 0. Just output Judement and don't output anything else.\n\n
"""
    return chat_template

def get_gpt4_score_ICE():
    example_1 = """
[Question]: Is the countertop tan or blue?
[Standard Answer]: A. The countertop is tan.
[Model_answer] : tan
Judgement: 1
""" # noqa

    example_2 = """
[Question]: On which side of the picture is the barrier?
[Standard Answer]: A. The barrier is on the left side of the picture.
[Model_answer] : A
Judgement: 1
""" # noqa

    example_3 = """
[Question]: Is the kite brown and large?
[Standard Answer]: A. Yes, the kite is brown and large.
[Model_answer] : Yes
Judgement: 1
""" # noqa

    example_4 = """
[Question]: Are the spots on a giraffe?
[Standard Answer]: A. No, the spots are on a banana.
[Model_answer] : no
Judgement: 1
""" # noqa

    example_5 = """
[Question]: Who is wearing pants?
[Standard Answer]: A. The boy is wearing pants.
[Model_answer] : C. The girl in the picture is wearing pants.
Judgement: 0
""" # noqa

    example_6 = """
[Question]: Is the man phone both blue and closed?
[Standard Answer]: A. Yes, the man phone is both blue and closed.
[Model_answer] : No.
Judgement: 0
""" # noqa

    example_7 = """
[Question]: What color is the towel in the center of the picture?
[Standard Answer]: A. The towel in the center of the picture is blue.
[Model_answer] : The towel in the center of the picture is pink.
Judgement: 0
""" # noqa

    return [example_1, example_2, example_3, example_4, example_5, example_6, example_7]

def get_prompt(predict_str, ground_truth, question):
    examples = get_gpt4_score_ICE()
    chat_template = get_chat_template()
    demo_prompt = chat_template
    for example in examples:
        demo_prompt += example + '\n\n'
    test_prompt = f"""
[Question]: {question}
[Standard Answer]: {ground_truth}
[Model_answer] : {predict_str}
Judgement:"""
    full_prompt = f'{demo_prompt}{test_prompt}'


    return full_prompt

In [25]:
result_root_path = '/cpfs/user/zhengziwei/workspace/agent/VeRL-Agent/eval_results/vlagent_think_mh/multiturn_all21k_0.8acc_-0.2format_0.4tool_4node_ds4_bugfree/global_step_48'
all_acc = []
per_type_acc = {}
per_type_tool_acc = {}
error_nums_type = {}
tool_mode = ['tool_acc', 'tool_wrong', 'notool_acc', 'notoool_wrong']
for test_type in test_types:
    per_type_acc[test_type] = []
    per_type_tool_acc[test_type] = {}
    error_nums_type[test_type] = 0
    for tm in tool_mode:
        per_type_tool_acc[test_type][tm] = []
for test_type in test_types:
    result_path = os.path.join(result_root_path, "result_"+test_type+'.jsonl')
    save_json = []
    with open(result_path, 'r') as f:
        lines = f.readlines()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        data = json.loads(line)
        question = data['question']
        answer = data['answer']
        pred_ans = data['pred_ans']
        pred_output = data['pred_output']
        answer = 'A. ' + answer

        if '\\boxed' in pred_ans:
            pred_ans = pred_ans.split('\\boxed{')[1].split('}')[0]

        

        # rule base check
        if len(pred_ans)==1:
            if pred_ans == 'A':
                acc_reward = 1.0
        elif len(pred_ans) == 2 and '.' in pred_ans:
            if 'A' in pred_ans:
                acc_reward = 1.0
        elif answer in pred_ans:
            acc_reward = 1.0
        else:
            full_prompt = get_prompt(pred_ans, answer, question)

            chat_response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": full_prompt},
                ],
                temperature=0.0,
            )
            response = chat_response.choices[0].message.content.strip()
            if 'ERROR' in pred_ans:
                error_nums_type[test_type] += 1
            # print (pred_ans)
            # print (answer)   
            # print ('acc', response)


            if 'Judgement:' in response:
                response = response.split('Judgement:')[-1].strip()
                if '1' in response:
                    acc_reward = 1.0
                elif '0' in response:
                    acc_reward = 0.0
                else:
                    print(f' [WARNING] resp format error {response=}')
                    acc_reward = 0.0
            else:
                if response == '1':
                    acc_reward = 1.0
                elif response == '0':
                    acc_reward = 0.0
                else:
                    print(f' [WARNING] resp format error {response=}')
                    acc_reward = 0.0

        acc = acc_reward
        all_acc.append(acc)
        per_type_acc[test_type].append(acc)
        
        tool_use = False
        for p_out in pred_output:
            if p_out['role'] == 'system' or p_out['role'] == 'user':
                continue
            p_content = p_out['content']
            if type(p_content) == str:
                p_content_msg = p_content.strip()
            elif type(p_content) == list:
                for _p_content in p_content:
                    if _p_content['type'] == 'text':
                        p_content_msg = _p_content['text']
            if '<tool_call>' in p_content_msg:
                tool_use = True

        if tool_use:
            if acc == 1.0:
                per_type_tool_acc[test_type]['tool_acc'].append(1.)
            else:
                per_type_tool_acc[test_type]['tool_wrong'].append(1.)
        else:
            if acc == 1.0:
                per_type_tool_acc[test_type]['notool_acc'].append(1.)
            else:
                per_type_tool_acc[test_type]['notoool_wrong'].append(1.)

        data['acc'] = acc
        
        save_json.append(data)
    
    with open(os.path.join(result_root_path, "result_"+test_type+'_tmp_acc.jsonl'), 'w') as f:
        for item in save_json:
            f.write(json.dumps(item) + '\n')
            

for test_type in test_types:
    print(f"Accuracy for {test_type}: {np.mean(per_type_acc[test_type]) * 100:.2f}%")
print(f"Overall Accuracy: {np.mean(all_acc) * 100:.2f}%")
for test_type in test_types:
    for tm in tool_mode:
        per_type_tool_acc[test_type][tm] = np.sum(per_type_tool_acc[test_type][tm])
print (per_type_tool_acc)
print (error_nums_type)

Accuracy for direct_attributes: 91.30%
Accuracy for relative_position: 84.21%
Overall Accuracy: 88.48%
{'direct_attributes': {'tool_acc': 105.0, 'tool_wrong': 10.0, 'notool_acc': 0.0, 'notoool_wrong': 0.0}, 'relative_position': {'tool_acc': 64.0, 'tool_wrong': 12.0, 'notool_acc': 0.0, 'notoool_wrong': 0.0}}
{'direct_attributes': 0, 'relative_position': 0}
